In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
# DEFINE DATA AS A DICTIONARY.
data_dict = {
    'userId': ['user 1', 'user 2', 'user 3', 'user 4', 'user 5'],
    'action1': [4, 5, 1, np.nan, 1],
    'action2': [5, 3, np.nan, 2, np.nan],
    'action3': [3, 3, np.nan, 1, 2],
    'romantic1': [np.nan, 2, 4, 4, 3],
    'romantic2': [2, 2, 5, np.nan, 3],
    'romantic3': [1, np.nan, 4, 3, 4]
}

# CREATE DATA-FRAME.
rating_data = pd.DataFrame(data_dict)
rating_data.set_index('userId', inplace=True)
rating_data.index.name = None

In [12]:
rating_data

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,NaN,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,NaN
user 3,1.0,NaN,NaN,4.0,5.0,4.0
user 4,NaN,2.0,1.0,4.0,NaN,3.0
user 5,1.0,NaN,2.0,3.0,3.0,4.0


#### Data Preprocessing

In [14]:
# STANDARIZE(NORMALIZE THE DATA AROUND ZERO) ALL THE RATINGS OF A USERS - MAKE 0 AS A MEAN
rating_data = rating_data.fillna(0) # ANY-WAYS ZERO IS GONNA BE MEAN.

def standardize(row):
    # MAKE 0 AS THE A MEAN(RATING).
    new_row = (row - row.mean()) / row.max()

    return new_row

In [15]:
rating_data = rating_data.apply(standardize)

In [16]:
rating_data

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40
